<a href="REF: https://github.com/GoogleCloudPlatform/df-ml-anomaly-detection/blob/master/README.md">Reference<a/> 

In [ ]:
gs://wesp-flow-logs/*.*

In [ ]:
"subscriberId":(11111111111,99999999999)
"srcIP": "ipv4"
"dstIP": "subnet"
"srcPort": (1000,5000)
"dstPort": (1000,5000)
"txBytes": (0,1000000000)
"rxBytes": (0,1000000000)
"tcpFlag": (0,65)
"startTime": "timestamp"
"endTime": "timestamp"
"protocolName": "tcp|udp|http"
"protocolNumber": (0,255)
"geoCountry": "country"
"geoCity": "city"
"latitude": (0,90)
"longitude": (0,180)

In [ ]:
"proto":6, "src":"100.111.22.33:21291", "dst":"100.111.44.55:63281",
"txPkts":2, "txBytes":108, "rxPkts":2, "rxBytes":112

In [ ]:
from faker import Faker
import geocoder
from random import choice

parser = argparse.ArgumentParser(__file__, description="Web Server Data Generator")
parser.add_argument("--num_users", "-u", type=int, dest="num_users",
                    help="The number of users to create", default=100)


faker = Faker()
# A list of functions for generating user agent strings for various browsers
ualist = [faker.firefox, faker.chrome, faker.safari, faker.internet_explorer, faker.opera]